In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" di|rectory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [43]:
from sklearn.model_selection import train_test_split
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv("../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

In [4]:
# defining features and label
X = df.review
y = df.sentiment

In [5]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.25, random_state=42)

In [6]:
# lowercase
def lower(string):
    return string.lower()

In [7]:
example=train_X.iloc[0]
example

'I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin because the writing just isn\'t clever. The movie just did not deliver enough laughs, twists, or tension to keep my interest. <br /><br />To be honest I did continue watching...Watching with hopes to see if anything suddenly clicked. It didn\'t. So, stylish as it is, I wouldn\'t recommend this movie. BTW, it seems odd to see Mary Tyler Moore as the principal. She\'s truly miscast, I hope the paycheck was inordinately big.'

In [8]:
#tokenizer = nltk.tokenize.TreebankWordTokenizer()
tokenizer = nltk.RegexpTokenizer(r"\w+")


In [9]:
def process(text,tokenizer=nltk.RegexpTokenizer(r"\w+"),stemmer=nltk.stem.PorterStemmer()):
    tokens = tokenizer.tokenize(text)
    result = " ".join(stemmer.stem(token) for token in tokens)
    result = result.lower()
    return result
    

In [10]:
tokenizer.tokenize(example)

['I',
 'figure',
 'this',
 'to',
 'be',
 'an',
 'alternate',
 'reality',
 'teen',
 'flick',
 'More',
 'precisely',
 'a',
 'Ferris',
 'Bueller',
 'type',
 'character',
 'as',
 'the',
 'leader',
 'of',
 'a',
 'cheat',
 'ring',
 'Yeah',
 'I',
 'know',
 'it',
 's',
 'not',
 'meant',
 'to',
 'compared',
 'to',
 'Ferris',
 'Bueller',
 'at',
 'least',
 'not',
 'in',
 'a',
 'oranges',
 'to',
 'oranges',
 'way',
 'but',
 'it',
 'will',
 'none',
 'the',
 'less',
 'br',
 'br',
 'Bottom',
 'line',
 'It',
 's',
 'galaxies',
 'away',
 'from',
 'even',
 'being',
 'even',
 'a',
 'minor',
 'classic',
 'It',
 'is',
 'watchable',
 'though',
 'only',
 'if',
 'you',
 're',
 'not',
 'expecting',
 'very',
 'much',
 'That',
 'said',
 'the',
 'main',
 'character',
 'has',
 'some',
 'charm',
 'but',
 'the',
 'premise',
 'wears',
 'thin',
 'because',
 'the',
 'writing',
 'just',
 'isn',
 't',
 'clever',
 'The',
 'movie',
 'just',
 'did',
 'not',
 'deliver',
 'enough',
 'laughs',
 'twists',
 'or',
 'tension',
 't

In [11]:
process(example)

'i figur thi to be an altern realiti teen flick more precis a ferri bueller type charact as the leader of a cheat ring yeah i know it s not meant to compar to ferri bueller at least not in a orang to orang way but it will none the less br br bottom line it s galaxi away from even be even a minor classic it is watchabl though onli if you re not expect veri much that said the main charact ha some charm but the premis wear thin becaus the write just isn t clever the movi just did not deliv enough laugh twist or tension to keep my interest br br to be honest i did continu watch watch with hope to see if anyth suddenli click it didn t so stylish as it is i wouldn t recommend thi movi btw it seem odd to see mari tyler moor as the princip she s truli miscast i hope the paycheck wa inordin big'

In [13]:
train_X = train_X.apply(lambda x : process(x))

In [14]:
test_X = test_X.apply(lambda x : process(x))

In [20]:
train_X = pd.DataFrame(train_X)
train_X

,review
27434,i figur thi to be an altern realiti teen flick...
13400,thi is the kind of movi that want to be good b...
883,thi wa by far the worst movi i ve ever seen an...
7303,aw aw aw br br i love the origin film it wa fu...
45124,fragil carn just befor hi great period althoug...
...,...
11284,shadow magic recaptur the joy and amaz of the ...
44732,i found thi movi to be quit enjoy and fairli e...
38158,avoid thi one it is a terribl movi so what if ...
860,thi product wa quit a surpris for me i absolut...


In [23]:
test_X = pd.DataFrame(test_X)
test_X

,review
33553,i realli like thi summerslam due to the look o...
9427,not mani televis show appeal to quit as mani d...
199,the film quickli get to a major chase scene wi...
12447,jane austen would definit approv of thi one br...
39489,expect were somewhat high for me when i went t...
...,...
9771,first separ the stori from the film the stori ...
34737,the most obviou flaw horribl horribl script th...
30390,the brillianc of thi movi is that even a compe...
6797,yaitat japan is a realli fun show and i realli...


In [24]:
vectorizer = TfidfVectorizer(min_df=0.1,max_df=0.5,ngram_range=(1,2))

In [25]:
X = vectorizer.fit_transform(train_X.review)

In [37]:
test_X = vectorizer.transform(test_X.review)

In [40]:
clf = MultinomialNB()

In [41]:
clf.fit(X,train_y)

MultinomialNB()

In [44]:
result = clf.predict(test_X)

In [ ]:
accuracy_score(test_y,result)